In [1]:
#Update your token
STUDENT_TOKEN = 'EUGENIO MARCHIORI'

## ignore this code, just used for submission
import requests
import pprint
import json
import random
import math
from copy import copy, deepcopy

def get_problem(problem_id):
  r = requests.get('https://emarchiori.eu.pythonanywhere.com/get-problem?TOKEN=%s&problem=%s' % (STUDENT_TOKEN, problem_id))
  if not r.status_code == 200:
    print('\033[91m' + str(r.status_code))
  return r.json()

def submit_answer(problem_id, answer):
  r = requests.get('https://emarchiori.eu.pythonanywhere.com/submit-answer?TOKEN=%s&problem=%s' % (STUDENT_TOKEN, problem_id), json = answer)
  if not r.status_code == 200:
    print('\033[91m' + str(r.status_code))
  result = r.json()['result']
  if result == 'PASSED':
    print('\033[92m' + result)
  else:
    print('\033[91m' + result)

In [2]:
def format_value(value):
  if value == '.':
    return '..'
  else:
    s = str(value)
    if len(s) == 1:
      s = ' ' + s
    return s

def format_h(const, max):
  s = ''
  for i in range(max - len(const)):
    s = s + '   '
  s = s + ' '.join(map(format_value, const))
  return s

def pretty_print(board, h_const, v_const):
  max_h = 1
  for c in h_const:
    max_h = max(max_h, len(c))
  max_v = 1
  for c in v_const:
    max_v = max(max_v, len(c))

  h_offset = ' '.join(map(lambda x: '  ', range(max_h)))
  for i in range(max_v):
    print('    ' + h_offset + ' '.join(map(lambda x: '  ' if i >= len(x) else format_value(x[i]), v_const)))
  for r in range(len(board)):
    row = board[r]
    print((h_offset if r ==0 else format_h(h_const[r-1], max_h)) + ' ' +  ' '.join(map(format_value, row)))

In [4]:
problem = get_problem('kakuro')

pprint.pprint(problem)

board = problem['board']
h_const = problem['h_constraints']
v_const = problem['v_constraints']

print('Input puzzle')
#pretty_print(board, h_const, v_const)

# This does not solve the problem, just makes it the same as the input
solution = board

# We print the solution in a way to make it easy to debug and check
#print('Solution')
#pretty_print(solution, h_const, v_const)

answer = {
    'board': board,
    'h_const': h_const,
    'v_const': v_const,
    'id': problem['id']
}

# The answer is submitted to the server, this is just to check the format (it does not check the solution itself)
# Note: We are doing this just to get used to it in following exercises mostly
submit_answer('kakuro', answer)

{'board': [['X', 'X', 'X', 'X', 'X'],
           ['X', '.', '.', '.', 'X'],
           ['X', '.', '.', '.', 'X'],
           ['X', 'X', '.', '.', '.'],
           ['X', 'X', '.', '.', '.']],
 'h_constraints': [[7], [19], [8], [15]],
 'id': 0,
 'v_constraints': [[5], [12], [22], [10]]}
Input puzzle
Ignore: not checking values yet


In [5]:
# Code Block 4: Representation of the problem

def initialize_domains(board):
    return {(row, col): list(range(1, 10)) for row in range(len(board)) for col in range(len(board[0])) if board[row][col] == '.'}

def initialize_constraints(board, h_const, v_const):
    constraints = {}
    # Populate the constraints dictionary with horizontal constraints
    for r, row in enumerate(h_const):
        sum_constraint = row[0]
        cells_in_block = [(r+1, c) for c, value in enumerate(board[r+1]) if value == '.']
        for cell in cells_in_block:
            constraints[cell] = constraints.get(cell, (0, 0))
            constraints[cell] = (sum_constraint, constraints[cell][1])

    # Populate the constraints dictionary with vertical constraints
    for c, col in enumerate(v_const):
        sum_constraint = col[0]
        cells_in_block = [(r, c+1) for r, value in enumerate(board) if value[c+1] == '.']
        for cell in cells_in_block:
            constraints[cell] = constraints.get(cell, (0, 0))
            constraints[cell] = (constraints[cell][0], sum_constraint)
    return constraints


In [6]:
#Code block 5: Constraint Propagation


# ensure that if a number is already assigned to a cell within a block, it cannot be assigned to another cell in the same block
def apply_unique_constraint(domains, constraints, cells_in_block):
    """Apply the unique values constraint to the domains of the cells in a block."""
    block_values = set()
    for cell in cells_in_block:
        if len(domains[cell]) == 1:
            block_values.add(domains[cell][0])
    for cell in cells_in_block:
        if len(domains[cell]) > 1:
            domains[cell] = [value for value in domains[cell] if value not in block_values]

# Code Block 5 (Continued): Implementing apply_sum_constraint

from itertools import combinations

# Precompute all possible combinations for each sum
possible_combinations = {}
for r in range(2, 10):  # From 2 to 9 inclusive
    for s in range(r, 45):  # Possible sum
        possible_combinations[(r, s)] = [combo for combo in combinations(range(1, 10), r) if sum(combo) == s]

def apply_sum_constraint(domains, constraints, cells_in_block, sum_constraint):
    """Apply the sum constraint to the domains of the cells in a block."""
    if len(cells_in_block) == 1:
        # If there's only one cell in the block, the domain is the sum itself if it's between 1 and 9.
        cell = cells_in_block[0]
        if 1 <= sum_constraint <= 9:
            domains[cell] = [sum_constraint] if sum_constraint in domains[cell] else []
        else:
            domains[cell] = []
        return

    # Get all possible combinations for the sum and number of cells in the block
    possible_sums = possible_combinations.get((len(cells_in_block), sum_constraint), [])
    
    # Intersect the possible sums with the current domains
    for i, cell in enumerate(cells_in_block):
        valid_values = set(val[i] for val in possible_sums)
        domains[cell] = [value for value in domains[cell] if value in valid_values]

def propagate_constraints(domains, constraints):
    """Apply constraints to reduce the domains of the cells."""
    for cell, (h_sum, v_sum) in constraints.items():
        h_cells = [c for c in constraints if c[0] == cell[0] and constraints[c][0] == h_sum]
        v_cells = [c for c in constraints if c[1] == cell[1] and constraints[c][1] == v_sum]
        apply_unique_constraint(domains, constraints, h_cells)
        apply_unique_constraint(domains, constraints, v_cells)
        apply_sum_constraint(domains, constraints, h_cells, h_sum)
        apply_sum_constraint(domains, constraints, v_cells, v_sum)

# Call the constraint propagation function to reduce the domains before starting the backtracking process.
propagate_constraints(domains, constraints)


In [8]:
# Code Block 6: Backtracking with a Stack


def is_valid_assignment(domains, constraints, cell, value):
    # Check if the value is unique in its block.
    row, col = cell
    h_sum, v_sum = constraints[cell]
    
    # Check horizontal block
    h_block_cells = [c for c in constraints if c[0] == row and constraints[c][0] == h_sum]
    if any(domains[c] == [value] for c in h_block_cells if c != cell):
        return False
    
    # Check vertical block
    v_block_cells = [c for c in constraints if c[1] == col and constraints[c][1] == v_sum]
    if any(domains[c] == [value] for c in v_block_cells if c != cell):
        return False
    
    # Check if the value does not violate the sum constraint.
    # For horizontal sum constraint
    if not is_sum_valid(domains, h_block_cells, h_sum, cell, value):
        return False
    
    # For vertical sum constraint
    if not is_sum_valid(domains, v_block_cells, v_sum, cell, value):
        return False
    
    return True

def is_sum_valid(domains, block_cells, sum_constraint, cell, value):
    # Calculate the sum of fixed values and the remaining possible sums for the block
    fixed_sum = value
    remaining_cells = []
    for c in block_cells:
        if c == cell:
            continue
        if len(domains[c]) == 1:
            fixed_sum += domains[c][0]
        else:
            remaining_cells.append(c)
    
    # Check if the fixed sum is already greater than the sum constraint
    if fixed_sum > sum_constraint:
        return False
    
    # Check if it is possible to reach the sum constraint with the remaining cells
    possible_sums = possible_combinations.get((len(remaining_cells), sum_constraint - fixed_sum), [])
    for combo in possible_sums:
        if all(value in domains[cell] for value, cell in zip(combo, remaining_cells)):
            return True
    
    return False


def select_unassigned_variable(domains, constraints):
    # Use the MRV heuristic to select the variable with the fewest legal values left.
    mrv_cell = None
    min_domain_size = float('inf')
    
    for cell, values in domains.items():
        if len(values) > 1 and len(values) < min_domain_size:  # Cell is unassigned and has fewer possibilities.
            min_domain_size = len(values)
            mrv_cell = cell
    
    return mrv_cell

def backtrack(domains, constraints):
    # Initialize the stack with the initial domains.
    stack = [(deepcopy(domains), [])]  # Each state in the stack is a tuple (domains, assignments)
    backtrack_count = 0

    while stack:
        current_domains, assignments = stack.pop()
        if all(len(values) == 1 for values in current_domains.values()):  # Solution found
            return assignments, backtrack_count

        # Select an unassigned variable using the MRV heuristic.
        cell = select_unassigned_variable(current_domains, constraints)
        if cell is None:
            continue  # No unassigned variables left, this path is a dead end.

        for value in current_domains[cell]:
            if is_valid_assignment(current_domains, constraints, cell, value):
                # Assign the value and push the new state onto the stack.
                new_domains = deepcopy(current_domains)
                new_domains[cell] = [value]  # Update the domain of the assigned cell.
                new_assignments = assignments + [(cell, value)]
                stack.append((new_domains, new_assignments))
            else:
                backtrack_count += 1  # Increment the backtrack count if the assignment is not valid.

    return None, backtrack_count  # No solution found.



In [9]:
# Code Block 7: Testing the Kakuro Solver

# Fetch a new Kakuro problem
problem = get_problem('kakuro')
print('Input puzzle:')
pprint.pprint(problem)

# Extract the board and constraints from the problem
board = problem['board']
h_const = problem['h_constraints']
v_const = problem['v_constraints']

# Initialize domains and constraints for the Kakuro puzzle
domains = initialize_domains(board)
constraints = initialize_constraints(board, h_const, v_const)

# Apply constraint propagation to reduce the domains
propagate_constraints(domains, constraints)

# Run the backtracking algorithm to find a solution
solution, backtrack_count = backtrack(domains, constraints)
print(f"Backtracking count: {backtrack_count}")

# If a solution is found, format it appropriately
if solution:
    print("Solution found:")
    for cell, value in solution:
        board[cell[0]][cell[1]] = value
    pretty_print(board, h_const, v_const)
    answer = {
        'board': board,
        'h_const': h_const,
        'v_const': v_const,


SyntaxError: incomplete input (1789685947.py, line 33)